In [1]:
import os
import time

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [2]:
# Load data
img = np.load(r"C:/Users/curti/Dropbox/Projects_all/479ML/Data/1000/sample_1000_image.npy")
lbl = np.load(r"C:/Users/curti/Dropbox/Projects_all/479ML/Data/1000/sample_1000_label.npy")

# Normalize image data.  0-255 to 0-1
img = img / 255
df = pd.DataFrame(np.concatenate((img, lbl), axis=1))

# Rename the last column as "label"
df.rename(columns={784:"label"}, inplace=True)

# Convert label column to float64 type
df['label'] = df['label'].astype('float64')

img = df.iloc[:, 0:-1].values
lbl = df['label'].values

#Split dataset
x_train1, x_test, y_train1, y_test = train_test_split(img, lbl, test_size = 0.20, random_state = 123, stratify = lbl)

#Convert dataframe to tensor
x_train1 = torch.tensor(x_train1).type('torch.FloatTensor')
y_train1 = torch.tensor(y_train1).type('torch.FloatTensor')

x_test = torch.tensor(x_test).type('torch.FloatTensor')
y_test = torch.tensor(y_test).type('torch.FloatTensor')

In [6]:
x_train1 = x_train1.reshape((-1,1,28,28))
x_test = x_test.reshape((-1,1,28,28))
print(x_train1.shape)
print(y_train1.shape)

torch.Size([276000, 1, 28, 28])
torch.Size([276000])


In [7]:
#Create dataloaders
BATCH_SIZE = 1000

train_dataset = TensorDataset(x_train1, y_train1)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=False)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

In [8]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

num_epochs = 2
for epoch in range(num_epochs):

    for batch_idx, (x, y) in enumerate(train_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

Epoch: 1 | Batch index: 0 | Batch size: 1000
Epoch: 2 | Batch index: 0 | Batch size: 1000


In [12]:
##########################
### SETTINGS
##########################

# Device
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
print('Device:', device)

# Hyperparameters
random_seed = 123
learning_rate = 0.005
num_epochs = 10

Device: cpu


In [13]:
##########################
### MODEL
##########################


class Autoencoder(torch.nn.Module):

    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # calculate same padding:
        # (w - k + 2*p)/s + 1 = o
        # => p = (s(o-1) - w + k)/2
        
        ### ENCODER
        
        # 28x28x1 => 28x28x4
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                      out_channels=4,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      # (1(28-1) - 28 + 3) / 2 = 1
                                      padding=1)
        # 28x28x4 => 14x14x4                              
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         # (2(14-1) - 28 + 2) / 2 = 0
                                         padding=0)                                       
        # 14x14x4 => 14x14x8
        self.conv_2 = torch.nn.Conv2d(in_channels=4,
                                      out_channels=8,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      # (1(14-1) - 14 + 3) / 2 = 1
                                      padding=1)                 
        # 14x14x8 => 7x7x8                             
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         # (2(7-1) - 14 + 2) / 2 = 0
                                         padding=0)
        
        ### DECODER
                                         
        # 7x7x8 => 14x14x8                          
        
        ## interpolation
        
        # 14x14x8 => 14x14x8
        self.conv_3 = torch.nn.Conv2d(in_channels=8,
                                      out_channels=4,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      # (1(14-1) - 14 + 3) / 2 = 1
                                      padding=1)
        # 14x14x4 => 28x28x4                            

        ## interpolation
        
        # 28x28x4 => 28x28x1
        self.conv_4 = torch.nn.Conv2d(in_channels=4,
                                      out_channels=1,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      # (1(28-1) - 28 + 3) / 2 = 1
                                      padding=1)
        
    def forward(self, x):
        
        ### ENCODER
        x = self.conv_1(x)
        x = F.leaky_relu(x)
        x = self.pool_1(x)
        x = self.conv_2(x)
        x = F.leaky_relu(x)
        x = self.pool_2(x)
        
        ### DECODER
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.conv_3(x)
        x = F.leaky_relu(x)
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = self.conv_4(x)
        x = F.leaky_relu(x)
        x = torch.sigmoid(x)
        return x

    
torch.manual_seed(random_seed)
model = Autoencoder()
model = model.to(device)
    

##########################
### COST AND OPTIMIZER
##########################

cost_fn = torch.nn.BCELoss() # torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
##########################
### TRAINING
##########################

epoch_start = 1


torch.manual_seed(random_seed)
model = Autoencoder()
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


################## Load previous
# the code saves the autoencoder
# after each epoch so that in case
# the training process gets interrupted,
# we will not have to start training it
# from scratch
files = os.listdir()

start_time = time.time()
for epoch in range(epoch_start, num_epochs+1):
    
    
    for batch_idx, (x, y) in enumerate(train_loader):

        # don't need labels, only the images (features)
        features = x.to(device)
        
        ### FORWARD AND BACK PROP
        decoded = model(features)
        cost = F.mse_loss(decoded, features)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 500:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch, num_epochs, batch_idx, 
                     len(train_loader), cost))

        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))
        
# Save model
if os.path.isfile('autoencoder_quickdraw-1_i_%d_%s.pt' % (epoch-1, device)):
    os.remove('autoencoder_quickdraw-1_i_%d_%s.pt' % (epoch-1, device))
torch.save(model.state_dict(), 'autoencoder_quickdraw-1_i_%d_%s.pt' % (epoch, device))

Epoch: 001/010 | Batch 0000/0276 | Cost: 0.2202


KeyboardInterrupt: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


model = Autoencoder()
model = model.to(device)
model.load_state_dict(torch.load('autoencoder_quickdraw-1_i_%d_%s.pt' % (num_epochs, device)))
model.eval()
torch.manual_seed(random_seed)

for batch_idx, (x, y) in enumerate(train_loader):
    features = x.to(device)
    decoded = model(features)
    break



##########################
### VISUALIZATION
##########################

n_images = 5

fig, axes = plt.subplots(nrows=2, ncols=n_images, 
                         sharex=True, sharey=True, figsize=(18, 5))
orig_images = features.detach().cpu().numpy()[:n_images]
orig_images = np.moveaxis(orig_images, 1, -1)

decoded_images = decoded.detach().cpu().numpy()[:n_images]
decoded_images = np.moveaxis(decoded_images, 1, -1)


for i in range(n_images):
    for ax, img in zip(axes, [orig_images, decoded_images]):
        ax[i].axis('off')
        ax[i].imshow(img[i].reshape(28, 28), cmap='binary')